In [11]:
import os
os.chdir('..')
import src.models_and_optimizers as model_utils
import yaml
from types import SimpleNamespace
from clip_main import get_wds_loaders
from transformers import EsmTokenizer
import src.data_utils as data_utils
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import torch.nn as nn
import torch
import sys
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import json
from torch.cuda.amp import autocast
from scipy.stats.stats import pearsonr 
import matplotlib.pyplot as plt
import webdataset as wds
import copy 
import time

/state/partition1/slurm_tmp/24926014.0.0/ipykernel_2608326/2538044701.py:20: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [2]:
## Load model parameters

model_dir = "/c/example/path"
path = os.path.join(model_dir, "checkpoints/checkpoint_latest.pt")
args_path = os.path.join(model_dir, 
                         [u for u in os.listdir(model_dir) if u.endswith('.pt')][0])

hparams = torch.load(args_path)
hparams['args']['blacklist_file'] = "pdb_blacklist.txt"
hparams['args']['coordinator_hparams'] = "terminator_configs/standard.json"
args_dict = hparams['args']
args_dict['batch_size'] = 1
args_dict['distributed'] = 0

args_dict['train_wds_path'] = 'wds_rocklin.tar'
args_dict['val_wds_path'] = 'wds_rocklin.tar'
args_dict['data_root'] = "example_data/wds"
args_dict['train_wds_path'] = 'wds_contact_test.tar'
args_dict['val_wds_path'] = 'wds_contact_test.tar'
args_dict['arch']= '/c/example/path'

args = SimpleNamespace(**args_dict)
coordinator_params = data_utils.get_coordinator_params(args.coordinator_hparams)
coordinator_params['num_positional_embeddings'] = args.gnn_num_pos_embs
coordinator_params['zero_out_pos_embs']= args.gnn_zero_out_pos_embs
coordinator_params['clip_mode'] = True


In [3]:
## Load model

trained_model = model_utils.load_model(path, args_dict['arch'], 'cuda')
tokenizer = EsmTokenizer.from_pretrained(args_dict['arch'])

loading state dict from /data1/groups/keating_madry/runs/new_blacklist/version_0/checkpoints/checkpoint_latest.pt
building model based on path


Some weights of the model checkpoint at /data1/groups/keating_madry/huggingface/esm2_t30_150M_UR50D were not used when initializing EsmModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing EsmModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmModel were not initialized from the model checkpoint at /data1/groups/keating_madry/huggingface/esm2_t30_150M_UR50D and are newly initialized: ['esm.pooler.dense.bias', 'esm.pooler.dense.weight']
You should probably TRAIN this model on a down-

ZERO OUT POS EMB False
GNN Potts Model Encoder hidden dimensionality is 128
freeze_llm False
{'training_args': {'epochs': 10, 'lr': 0.001, 'weight_decay': 0.001, 'momentum': 0.9, 'label_smoothing': 0.0, 'lr_peak_epoch': 2, 'eval_epochs': 2, 'only_non_bn_weight_decay': False, 'opt': 'ADAM', 'lr_scheduler': 'cosine', 'burn_in': 0, 'mixed_precision': True, 'max_len': 1024, 'self_supervised': True}, 'epoch': 9, 'training_metrics': {'loss': 0.6333152562079322, 'acc': 0.6356185298161776}, 'val_metrics': {'loss': 1.6227540006807477, 'acc': 0.5308778800395427}, 'model_building_args': {'esm_arch': 'facebook/esm2_t30_150M_UR50D', 'terminator_hparams': {'cov_features': 'all_raw', 'cov_compress': 'project', 'term_use_mpnn': True, 'matches': 'transformer', 'energies_style': 'mpnn', 'energies_use_mpnn': True, 'energies_full_graph': True, 'contact_idx': True, 'energies_encoder_layers': 3, 'energies_hidden_dim': 128, 'resnet_linear': True, 'matches_linear': True, 'transformer_linear': True, 'term_mpnn

In [4]:
## Functions to extract attention "contact" values from ESM-RLA

import torch
from torch.cuda.amp import GradScaler, autocast
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence

def get_formatted_inp(batch):
    seq_batch, coords_batch = batch
    seqs = seq_batch['string_sequence']
    text_inp = tokenizer(seqs, return_tensors='pt', padding=True, 
                              truncation=True, max_length=1024+2)
    text_inp['position_ids'] = seq_batch['pos_embs'][0]
    text_inp = {k: v.to('cuda') for k, v in text_inp.items()}
    return text_inp, seq_batch, coords_batch

def postprocess_attention_features(attention_features, inp_dict, tokenizer, placeholder_mask):
    # remove class token, eos token
    attention_features, text_mask = _adjust_text_features(attention_features, inp_dict, tokenizer)
    # remove placeholders
    attention_features, text_mask = _remove_placeholders(attention_features, text_mask, placeholder_mask)
    return attention_features, text_mask


def _adjust_text_features(attention_features, inp_dict, tokenizer):
    mask = inp_dict['attention_mask'].clone()
    toks = inp_dict['input_ids']
    eos_token = tokenizer.convert_tokens_to_ids(tokenizer.eos_token)
    mask[toks == eos_token] = 0
    # ignore class token and last eos token
    mask = mask[:, 1:-1]
    attention_features = attention_features[:, :, 1:-1, 1:-1]
    return attention_features, mask

def _remove_placeholders(attention_features, text_mask, placeholder_mask):
    B = placeholder_mask.shape[0]
    filtered = []
    new_masks = []
    for b in range(B):
        p_m = placeholder_mask[b]
        new_text_feat = attention_features[b][:, p_m]
        new_text_feat = new_text_feat[:, :, p_m]
        filtered.append(new_text_feat)
        new_masks.append(text_mask[b][p_m])
    new_masks = pad_sequence(new_masks, batch_first=True)
    dest_shape = 1024
    padded_filtered = []
    for f in filtered:
        amt_to_pad = dest_shape - f.shape[-1]
        padded = torch.nn.functional.pad(f, (0, amt_to_pad, 0, amt_to_pad))
        padded_filtered.append(padded)
    return torch.stack(padded_filtered), new_masks

def get_attentions(model, text_inp_):
    with torch.no_grad():
        out = model.text_model(**text_inp_, output_attentions=True)
        all_attns = torch.cat(out['attentions'], 1)
        num_heads = len(out['attentions'])
    attns = all_attns.cpu()
    return attns, num_heads

def APC_correction(attn_):
    F_i = attn_.sum(keepdims=True, dim=2)
    F_j = attn_.sum(keepdims=True, dim=3)
    F = attn_.sum(dim=2, keepdims=True).sum(dim=3, keepdims=True)
    apc_corr = (F_i * F_j)/F
    attn_corr = attn_ - apc_corr
    return attn_corr

def pred_and_contact_gt(model, batch_):
    text_inp, seq_batch, coord_batch = get_formatted_inp(batch_)
    mask = seq_batch['seq_loss_mask'][0]
    mask = torch.nn.functional.pad(mask, (0, 1024 - mask.shape[-1]))
    B = mask.shape[0]
    T = mask.shape[-1]
    # get 2D mask
    M = mask.int()
    
    M = M.unsqueeze(-1) @ M.unsqueeze(1)
    local_mask = torch.ones(T, T)
    for i in range(6):
        torch.diagonal(local_mask, i).zero_()
        torch.diagonal(local_mask, i*-1).zero_()
    
    # M[:, local_mask == 0] = 0 
    M = M==1
    # get index distance
    U = torch.arange(T).float().unsqueeze(1)
    U = torch.cdist(U, U).expand((B, T, T))
    
    # get indices
    U1 = torch.arange(T).unsqueeze(1)
    U1 = U1.expand((B, T, T))
    
    U2 = torch.arange(T).unsqueeze(0)
    U2 = U2.expand((B, T, T))
    
    # get distances and contacts
    coords = coord_batch['coords'][0][:, :, 1]
    dists = torch.cdist(coords, coords)
    contacts = dists < 8
    amt_to_pad = 1024 - dists.shape[-1] 
    dists = torch.nn.functional.pad(dists, (0, amt_to_pad, 0, amt_to_pad))
    contacts = torch.nn.functional.pad(contacts, (0, amt_to_pad, 0, amt_to_pad))
    
    
    # get attn
    attns, num_heads = get_attentions(model, text_inp)
    attns, _ = postprocess_attention_features(attns, text_inp, tokenizer, seq_batch['placeholder_mask'][0])
    attns.permute(0, 2, 3, 1)[~M] = 0
    attns = APC_correction(attns)
    attns = attns.permute(0, 2, 3, 1)
    return {
        'X_': attns,
        'Y_': contacts,
        'num_heads': num_heads,
        'dists': dists,
        'M': M,
        'U': U,
        'U1': U1,
        'U2': U2,
    }

In [5]:
## Pass data through ESM_RLA to be able to get attention "contact" scores
def get_X_and_Y(models, loader, num_iters=5):
    g_cpu = torch.Generator()
    g_cpu.manual_seed(10)
    OUTPUT_KEYS = ['X_', 'Y_', 'dists', 'U', 'U1', 'U2']
    output = {
        o: {k: [] for k in models.keys()} 
        for o in OUTPUT_KEYS
    }
    for batch_idx, batch_ in enumerate(tqdm(loader)): 
        train_indices = None
        if torch.sum(batch_[0]['placeholder_mask'][0]) < 30:
            continue
        for k in models.keys():
            model = models[k]
            result = pred_and_contact_gt(model, batch_)
            t2 = time.time()
            result_subset = {o: result[o][result['M']] for o in OUTPUT_KEYS}
            if train_indices is None:
                pos_contacts = torch.where(result_subset['Y_'] == 1)[0]
                neg_contacts = torch.where(result_subset['Y_'] == 0)[0]
                neg_contacts = neg_contacts[torch.randperm(len(neg_contacts), generator=g_cpu)[:len(pos_contacts)]]
                train_indices = torch.cat([pos_contacts, neg_contacts])
            for o in OUTPUT_KEYS:
                output[o][k].append(result_subset[o][train_indices])
            t3 = time.time()
        if batch_idx == num_iters:
            break
            
            
    output = {
        o: {k: torch.cat(v) for k, v in output_sub.items()}
        for o, output_sub in output.items()
    }
    
    
    
    return output

In [6]:
## Load sample data
_, val_loader, _, _ = get_wds_loaders(args, coordinator_params, gpu=None, shuffle_train=False, val_only=False, return_count=False)

{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': -1, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
{'max_coords_len': 2000, 'shuffle_coords': False, 'max_seq_len': 1024, 'pos_offset': 128, 'burn_in': 0, 'k_neighbors': 30, 'crop_type': 'absolute', 'shuffle_chains': False, 'num_mutations': -1, 'masked_rate': -1.0, 'masked_mode': 'MASK'}
added select filtering... 30
0 0


In [7]:
models = {
    'trained': trained_model,
}
val_output = get_X_and_Y(models, val_loader, num_iters=20)

20it [00:29,  1.50s/it]


In [8]:
val_X, val_Y, val_dists, val_U = val_output['X_'], val_output['Y_'], val_output['dists'], val_output['U']

In [26]:
## Create top layer to predict contacts from attention
def gelu(x):
    """
    This is the gelu implementation from the original ESM repo. Using F.gelu yields subtly wrong results.
    """
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class MultiLayerLinear(nn.Module):
    def __init__(self, in_features, out_features, num_layers, activation_layers='relu', dropout=0):
        super(MultiLayerLinear, self).__init__()
        self.activation_layers = activation_layers
        self.layers = nn.ModuleList()
        for i in range(num_layers):
            self.layers.append(nn.Linear(in_features[i], out_features[i]))
        self.dropout_prob = dropout
        if dropout > 0:
            self.dropout = nn.Dropout(dropout)
            
    def forward(self, x):
        for layer in self.layers:  
            x = layer(x)
            if self.activation_layers == 'relu':
                x = nn.functional.relu(x)
            else:
                x = gelu(x)
        if self.dropout_prob > 0:
            x = self.dropout(x)
        return x
    
    
def train_finetune_model(X_, Y_, U_, D_, crit='BCE'):
    inp_dim = 600

    finetune_model = MultiLayerLinear(in_features=[600,100], out_features=[100,1], num_layers=2, activation_layers='gelu', dropout=0).cuda()

    # finetune_model = nn.Linear(inp_dim, 1).cuda()
    
    if crit == 'BCE':
        criterion = nn.BCEWithLogitsLoss()
    else:
        criterion = nn.MSELoss()
    
    optimizer = FullBatchLBFGS(finetune_model.parameters(), lr=0.01)
    print('loaded data')
    optimizer.zero_grad()
    out = finetune_model(X_).squeeze(1)
    loss = criterion(out, Y_)
    loss.backward()

    epochs=300
    pbar = tqdm(range(epochs))
    for epoch in pbar:
        def closure():
            optimizer.zero_grad()
            out = finetune_model(X_).squeeze(1)
            if crit=='BCE':
                loss = criterion(out, Y_)
            else:
                loss = criterion(out, D_)
            return loss
        options = {'closure': closure, 'current_loss': loss}
        loss = optimizer.step(options)[0]
        pbar.set_postfix({'loss': loss.item()})
    return finetune_model

def metric_fn(pred, gt, crit='BCE'):
    frac_neg = (gt[0] == 0).float().mean()
    frac_pos = (gt[0] == 1).float().mean()
    if crit == 'BCE':
        neg = (pred[gt[0] == 0] == 0).float().mean().item()
        pos = (pred[gt[0] == 1] == 1).float().mean().item()
        total = (gt[0] == pred).float().mean().item()
    else:
        pred = pred.squeeze(1)
        neg = ((pred[gt[0] == 0] - gt[1][gt[0] == 0])**2).float().mean().item()
        pos = ((pred[gt[0] == 1] - gt[1][gt[0] == 1])**2).float().mean().item()
        total = ((pred - gt[1])**2).float().mean().item()
    print(f"All -> {total:0.4f} \t Neg ({frac_neg*100:0.2f}%) -> {neg:0.4f} \t Pos({frac_pos*100:0.2f}%) -> {pos:0.4f}")
    
def get_U_mask(U, min_, max_):
    return (U > min_) & (U <= max_)

def evaluate(finetune_model, X_, Y_, U, D, crit='BCE'):
    with torch.no_grad():
        out = finetune_model(X_)
        if crit == 'BCE':
            preds = (nn.Sigmoid()(out.squeeze(1)) > 0.5).float()
        else:
            preds = out.float()
        print("overall")
        metric_fn(pred=preds, gt=(Y_, D), crit=crit)
        
        mask = get_U_mask(U, 6, 12)
        print("short", mask.sum().item())
        metric_fn(pred=preds[mask], gt=(Y_[mask], D[mask]), crit=crit)
        
        mask = get_U_mask(U, 12, 24)
        print("medium", mask.sum().item())
        metric_fn(pred=preds[mask], gt=(Y_[mask], D[mask]), crit=crit)
        
        mask = get_U_mask(U, 24, np.inf)
        print("long", mask.sum().item())
        metric_fn(pred=preds[mask], gt=(Y_[mask], D[mask]), crit=crit)
    return preds.cpu()

def evaluate_ft(finetune_model, scaler, MODEL_TO_ANALYZE):
    val_X_, val_Y_, val_D_ = val_X[MODEL_TO_ANALYZE], val_Y[MODEL_TO_ANALYZE], val_dists[MODEL_TO_ANALYZE]
    val_X_norm = torch.tensor(scaler.transform(val_X_.numpy()))
    val_X_norm, val_Y_norm = val_X_norm.cuda(), val_Y_.cuda().float()
    U = val_output['U'][MODEL_TO_ANALYZE]
    val_preds = evaluate(finetune_model, val_X_norm, val_Y_norm, U, val_D_)
    return val_preds

In [27]:
from joblib import load

In [28]:
# evaluate
contact_model_ = nn.Linear(600, 1).cuda()
contact_model_.load_state_dict(torch.load(os.path.join(os.getcwd(), "contact_head/finetune.pt")))
contact_scaler = load(os.path.join(os.getcwd(), "contact_head/finetune_scaler.bin"))

/home/gridsan/fbirnbaum/.local/lib/python3.9/site-packages/sklearn/base.py:288: UserWarning: Trying to unpickle estimator StandardScaler from version 1.2.2 when using version 1.2.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [29]:
evaluate_ft(contact_model_, contact_scaler, 'trained')

overall
All -> 0.7318 	 Neg (50.00%) -> 0.9314 	 Pos(50.00%) -> 0.5323
short 7388
All -> 0.8499 	 Neg (36.55%) -> 0.6352 	 Pos(63.45%) -> 0.9735
medium 14651
All -> 0.9183 	 Neg (34.12%) -> 0.8062 	 Pos(65.88%) -> 0.9764
long 124991
All -> 0.9142 	 Neg (80.23%) -> 0.9505 	 Pos(19.77%) -> 0.7668


tensor([0., 0., 0.,  ..., 1., 1., 0.])